In [1]:
import numpy as np
import matplotlib as plt
from hilp import load_csv_data
from helper import *
from costs import *
from least_squares import *
from ridge_regression import *
from logistic_regression import *

In [2]:
yb, input_data, ids = load_csv_data('data/train.csv')

In [3]:
input_data.shape

(250000, 30)

In [4]:
cleaned_columns = [c for c in input_data.T if (c==-999).sum()/len(c) < 0.2]
print(np.shape(cleaned_columns))
for c in cleaned_columns:
    numb_of_nan = (c==-999).sum()
    median = np.median(list(filter(lambda x : x!= -999, c)))
    c[c == -999] = median

(20, 250000)


In [5]:
cleaned_columns

[array([138.47 , 160.937, 112.406, ..., 105.457,  94.951, 112.406]),
 array([ 51.655,  68.768, 162.172, ...,  60.526,  19.362,  72.756]),
 array([ 97.827, 103.235, 125.953, ...,  75.839,  68.812,  70.831]),
 array([27.98 , 48.146, 35.635, ..., 39.757, 13.504,  7.479]),
 array([3.064, 3.473, 3.148, ..., 2.39 , 3.365, 2.025]),
 array([41.928,  2.078,  9.336, ..., 22.183, 13.504,  7.479]),
 array([197.76 , 125.157, 197.814, ..., 120.462,  55.859,  83.24 ]),
 array([1.582, 0.879, 3.776, ..., 1.202, 0.999, 0.936]),
 array([ 1.396,  1.414,  1.414, ...,  0.529,  1.414, -1.411]),
 array([32.638, 42.014, 32.154, ..., 35.636, 27.944, 43.003]),
 array([ 1.017,  2.039, -0.705, ..., -0.266, -2.211,  1.685]),
 array([ 0.381, -3.011, -2.093, ..., -3.132,  2.792,  2.653]),
 array([ 51.626,  36.918, 121.409, ...,  42.834,  27.915,  40.236]),
 array([ 2.273,  0.501, -0.953, ...,  0.381, -0.874,  1.49 ]),
 array([-2.414,  0.103,  1.052, ...,  0.851, -0.296,  0.637]),
 array([16.824, 44.704, 54.283, ..., 

In [6]:
initial_w = np.random.randint(0,1,len(cleaned_columns))
batch_size = 1
max_iters = 20
#gamma = 0.01
lambda_ = 0.01

In [7]:
gammas = [0.001, 0.002, 0.005, 0.01, 0.02, 0.03]

In [8]:
# standardize the data
std_data, mean, std = standardize(np.transpose(cleaned_columns))

In [9]:
# build the model (add 1's for the w0)
tx, y = build_model_data(std_data, yb)

In [ ]:
l = []
W = []
for gamma in gammas:
    loss, ws = gradient_descent(y, tx, initial_w, max_iters, gamma)
    l.append(loss[max_iters-1])
    W.append(ws)
best_loss = min(l)
ind = l.index(best_loss)
print("best loss :" + str(best_loss)+ " " + str(gammas[ind]) +" \n "+ str(W[ind]))

In [12]:
stochastic_gradient_descent(y, tx, initial_w, batch_size, max_iters, gamma)

SGD iter. 0/19: loss=0.5, w0=0.03, w1=0.03017918643569754
SGD iter. 1/19: loss=0.9229100967786545, w0=-0.009705064918780183, w1=-0.004663240736245754
SGD iter. 2/19: loss=0.6399740929166161, w0=-0.002661443954838913, w1=0.0036358678329791016
SGD iter. 3/19: loss=0.7645522196061143, w0=0.006098844224092026, w1=0.014351063696840338
SGD iter. 4/19: loss=0.9041656377049222, w0=0.022515681858204153, w1=0.03440532353838178
SGD iter. 5/19: loss=1.1171880488022574, w0=-0.01027217158835007, w1=0.005245185109580022
SGD iter. 6/19: loss=0.9223599870931383, w0=0.015437015269327371, w1=0.02780579475902592
SGD iter. 7/19: loss=1.1910645413632905, w0=-0.013368068346780511, w1=0.0025284395871102427
SGD iter. 8/19: loss=1.0458314872351304, w0=-0.05908932583930209, w1=-0.0497475569083414
SGD iter. 9/19: loss=0.5407527197873756, w0=-0.11157884554518674, w1=-0.06331782154975882
SGD iter. 10/19: loss=0.7819871023513536, w0=-0.04876698401679484, w1=0.04302768727108648
SGD iter. 11/19: loss=5.172503282661993

([0.5,
  0.9229100967786545,
  0.6399740929166161,
  0.7645522196061143,
  0.9041656377049222,
  1.1171880488022574,
  0.9223599870931383,
  1.1910645413632905,
  1.0458314872351304,
  0.5407527197873756,
  0.7819871023513536,
  5.172503282661993,
  4.640593133892575,
  3.2478541097268745,
  3.8442651672131416,
  3.5254021363072137,
  1.5993450971762317,
  0.7640233116621199,
  0.7682807018754992,
  0.9304503932592871],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([ 0.03      ,  0.03017919, -0.01049198,  0.00167854,  0.06166404,
         -0.0171028 , -0.00703534,  0.08554633, -0.01734664, -0.01739783,
          0.00023626, -0.01780812, -0.01745996, -0.00615184, -0.01829813,
         -0.01752521,  0.03283282, -0.01740916,  0.14134448, -0.01681599,
          0.05626691]),
  array([-0.00970506, -0.00466324, -0.02956234, -0.04475465,  0.06534441,
          0.00422022,  0.01594262,  0.06132705,  0.00520777,  0.00638847,
          0.01188256,  0.00624969,

In [13]:
least_squares(y, tx)

(array([ 2.95137215e+02,  1.64125594e-02, -5.66315112e-01, -5.06641813e-01,
         4.81722734e-03,  2.78021755e+01, -1.39849015e-01, -2.39440384e+02,
        -1.76046142e+01,  8.44767115e+00,  2.40099930e+02, -5.66366907e-02,
        -3.61739715e-02,  2.40467040e+02, -3.75756920e-02,  9.84796412e-02,
         2.46397753e-01,  3.50785025e-02, -4.26087024e-02,  6.56880260e+00,
         2.39396437e+02]),
 0.3524927958636052)

In [14]:
ridge_regression(y, tx, lambda_)

array([-0.08441931,  0.07796364, -0.63864758, -0.09861935,  0.07178693,
        0.08681586, -0.07426362,  0.14507038,  0.02372936,  0.1226225 ,
        0.37526345,  0.04789182,  0.04381124,  0.01369728,  0.04816825,
        0.05241047,  0.05438705,  0.04991389, -0.03574803,  0.0640392 ,
       -0.14485184])

In [15]:
logistic_regression(y, tx, initial_w, max_iters, gamma)

Logistic regression iter. 0/19: loss=-48754.730509484805, w0=-0.048879839999999994, w1=-0.04695973632452501
Logistic regression iter. 1/19: loss=-35707.24449313245, w0=-0.08985714268353152, w1=-0.08554133505336953
Logistic regression iter. 2/19: loss=-27400.740608232303, w0=-0.12557639870454806, w1=-0.11870400741611611
Logistic regression iter. 3/19: loss=-21668.50599601481, w0=-0.15768922238301664, w1=-0.14825707584077008
Logistic regression iter. 4/19: loss=-17495.7892064634, w0=-0.1871999402570617, w1=-0.17527031963985346
Logistic regression iter. 5/19: loss=-14342.976646977268, w0=-0.21475873152825634, w1=-0.20041883217887124
Logistic regression iter. 6/19: loss=-11894.873501409907, w0=-0.2408104444433073, w1=-0.2241542070951057
Logistic regression iter. 7/19: loss=-9954.147865209428, w0=-0.2656728388295577, w1=-0.2467926323454867
Logistic regression iter. 8/19: loss=-8390.53641002046, w0=-0.2895806336480255, w1=-0.26856351577779847
Logistic regression iter. 9/19: loss=-7114.384379

([-48754.730509484805,
  -35707.24449313245,
  -27400.740608232303,
  -21668.50599601481,
  -17495.7892064634,
  -14342.976646977268,
  -11894.873501409907,
  -9954.147865209428,
  -8390.53641002046,
  -7114.384379808029,
  -6061.870217468984,
  -5186.282445945543,
  -4452.6291641406415,
  -3834.1773874662094,
  -3310.1580823805384,
  -2864.2010657928963,
  -2483.2412669871364,
  -2156.7377523566697,
  -1876.105290579835,
  -1634.2934553523055],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([-0.04887984, -0.04695974, -0.01189169, -0.02343984,  0.00090263,
          0.02716833,  0.01637539, -0.05904454,  0.02763437,  0.02899441,
          0.00793917,  0.02867824,  0.02867137, -0.00155803,  0.02868595,
          0.02864994,  0.00266014,  0.0286886 , -0.09218265,  0.02814552,
         -0.00808147]),
  array([-0.08985714, -0.08554134, -0.02352073, -0.04277793,  0.00520394,
          0.04993495,  0.0304316 , -0.10193141,  0.05078451,  0.05336131,
        

In [16]:
reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma)

Reg Logistic regression iter. 0/19: loss=-48754.7305094843, w0=-0.048879839999999994, w1=-0.04695973632452501
Reg Logistic regression iter. 1/19: loss=-35707.24450984966, w0=-0.08985714262487571, w1=-0.08554133499701785
Reg Logistic regression iter. 2/19: loss=-27400.74064238416, w0=-0.12557639854520963, w1=-0.11870400726436083
Reg Logistic regression iter. 3/19: loss=-21668.506045273127, w0=-0.1576892220884089, w1=-0.14825707556179624
Reg Logistic regression iter. 4/19: loss=-17495.7892680503, w0=-0.18719993979626884, w1=-0.17527031920520758
Reg Logistic regression iter. 5/19: loss=-14342.976718272328, w0=-0.2147587308723477, w1=-0.2004188315618374
Reg Logistic regression iter. 6/19: loss=-11894.873580093541, w0=-0.2408104435644833, w1=-0.22415420626992782
Reg Logistic regression iter. 7/19: loss=-9954.147949277789, w0=-0.2656728377006816, w1=-0.24679263128696324
Reg Logistic regression iter. 8/19: loss=-8390.536497763096, w0=-0.2895806322423549, w1=-0.26856351446106036
Reg Logistic r

([-48754.7305094843,
  -35707.24450984966,
  -27400.74064238416,
  -21668.506045273127,
  -17495.7892680503,
  -14342.976718272328,
  -11894.873580093541,
  -9954.147949277789,
  -8390.536497763096,
  -7114.384469776403,
  -6061.870308444927,
  -5186.282536912325,
  -4452.629254257233,
  -3834.1774760446015,
  -3310.158168865751,
  -2864.2011497453364,
  -2483.241348066989,
  -2156.7378303100504,
  -1876.1053652265157,
  -1634.2935265748158],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([-0.04887984, -0.04695974, -0.01189169, -0.02343984,  0.00090263,
          0.02716833,  0.01637539, -0.05904454,  0.02763437,  0.02899441,
          0.00793917,  0.02867824,  0.02867137, -0.00155803,  0.02868595,
          0.02864994,  0.00266014,  0.0286886 , -0.09218265,  0.02814552,
         -0.00808147]),
  array([-0.08985714, -0.08554133, -0.02352073, -0.04277793,  0.00520394,
          0.04993495,  0.0304316 , -0.10193141,  0.05078451,  0.05336131,
          0

In [10]:
from cross_validation import * 
degree = 3
k_fold = 4
lambdas = np.logspace(-7,0,8)
degrees = [3,5,7]
seed = 1
k_indices = build_k_indices(y, k_fold, seed)


In [26]:
initial_w = np.random.randint(0,1,len(tx.T))
max_iters = 30
gammas = np.logspace(-4,-0.5,25)

In [27]:
print(initial_w.shape)

(21,)


In [ ]:
cross_validation_gradient_descent(y, tx,'gradient_descent', k_fold, initial_w=initial_w, max_iters = max_iters, gammas = gammas)

1
2
3
3,5
GD iter. 0/29: loss=0.5, w0=-3.14664e-05, w1=-2.917213486366231e-05
GD iter. 1/29: loss=0.4999155262016901, w0=-6.290274820779291e-05, w1=-5.831208772346679e-05
GD iter. 2/29: loss=0.49983125344948176, w0=-9.430908779251208e-05, w1=-8.741990469786576e-05
GD iter. 3/29: loss=0.49974718112607414, w0=-0.00012568546185444371, w1=-0.00011649563183189116
GD iter. 4/29: loss=0.4996633086161607, w0=-0.0001570319134251389, w1=-0.000145539315097274
GD iter. 5/29: loss=0.4995796353064226, w0=-0.00018834848546752563, w1=-0.00017455100039256375
GD iter. 6/29: loss=0.4994961605855226, w0=-0.00021963522087602047, w1=-0.00020353073354324752
GD iter. 7/29: loss=0.49941288384409843, w0=-0.00025089216247664, w1=-0.0002324785603018687
GD iter. 8/29: loss=0.4993298044747557, w0=-0.0002821193530271124, w1=-0.0002613945263481459
GD iter. 9/29: loss=0.4992469218720626, w0=-0.00031331683521698846, w1=-0.00029027867728909155
GD iter. 10/29: loss=0.49916423543254296, w0=-0.00034448465166775263, w1=-0.0

GD iter. 14/29: loss=0.49883543930476454, w0=-0.0004688601101332583, w1=-0.0004342237965847284
GD iter. 15/29: loss=0.4987537246969702, w0=-0.0004998802348361552, w1=-0.0004629180105842749
GD iter. 16/29: loss=0.4986722026722048, w0=-0.0005308709481056471, w1=-0.0004915807267017148
GD iter. 17/29: loss=0.49859087264050367, w0=-0.0005618322920898468, w1=-0.0005202119899666535
GD iter. 18/29: loss=0.4985097340138082, w0=-0.0005927643088696826, w1=-0.0005488118453370483
GD iter. 19/29: loss=0.4984287862059583, w0=-0.000623667040459007, w1=-0.0005773803376993247
GD iter. 20/29: loss=0.4983480286326867, w0=-0.0006545405288047063, w1=-0.0006059175118684931
GD iter. 21/29: loss=0.4982674607116129, w0=-0.0006853848157868097, w1=-0.0006344234125882646
GD iter. 22/29: loss=0.4981870818622365, w0=-0.0007161999432185978, w1=-0.0006628980845311675
GD iter. 23/29: loss=0.4981068915059321, w0=-0.0007469859528467117, w1=-0.0006913415722986629
GD iter. 24/29: loss=0.498026889065942, w0=-0.0007777428863